# NER vs LLM

---

### Configure OLLama

Install OLLama

Open terminal and type:

`ollama run SpeakLeash/bielik-11b-v2.3-instruct:Q4_K_M`

Then set up chat with ollama:

In [1]:
import json

import pandas as pd
import requests


def chat_ollama(text):
    url = "http://localhost:11434/api/chat"

    payload = {
        "model": "SpeakLeash/bielik-11b-v2.3-instruct:Q4_K_M",
        "messages": [{"role": "user", "content": text}],
        "stream": False,
    }

    response = requests.post(url, json=payload)

    # Parse the response
    if response.status_code == 200:
        return response.json()["message"]["content"]

    return None


question = "Możesz opowiedzieć mi o problemie Named Entity Recognition?"

chat_ollama(question)

'Named Entity Recognition (NER) to zadanie przetwarzania języka naturalnego, które polega na identyfikacji i klasyfikacji nazwanych jednostek w tekście. Nazwane jednostki to takie elementy jak osoby, organizacje, lokalizacje, daty, ilości itp., które mają specjalne znaczenie dla kontekstu i mogą być użyteczne do dalszej analizy tekstu.\n\nProblem NER polega na tym, że teksty w języku naturalnym są często niejednoznaczne i zawierają wiele słów, które mogą mieć różne znaczenia w zależności od kontekstu. Na przykład, słowo "John" może być imieniem osoby lub nazwą miejsca. Zadaniem NER jest rozpoznanie tych jednostek i przypisanie im odpowiednich etykiet (np. osoba, organizacja, lokalizacja itp.).\n\nProblem ten jest trudny z kilku powodów:\n\n1. **Kontekst**: Słowa mogą mieć różne znaczenia w zależności od kontekstu, w którym są używane. Na przykład, słowo "bank" może oznaczać instytucję finansową lub brzeg rzeki.\n2. **Niejednoznaczność**: Niektóre słowa mogą być nazwanymi jednostkami w 

Using `generate` endpoint

In [2]:
url = "http://localhost:11434/api/generate"


def prompt_ollama(text):
    payload = {
        "model": "SpeakLeash/bielik-11b-v2.3-instruct:Q4_K_M",
        "prompt": text,
        "context": [],
        "options": {"top_k": 10, "temperature": 0},
        "stream": False,
    }

    response = requests.post(url, data=json.dumps(payload))

    # Parse the response
    if response.status_code == 200:
        return response.json()["response"]

    return None

In [4]:
prompt_ollama("Czy potrafisz mówić po polsku?")

'Tak, potrafię mówić po polsku. W czym mogę Ci pomóc?'

### Take 1000 passages from fiqa corpus

In [5]:
from datasets import load_dataset
from numpy.random import choice

fiqa_corpus = load_dataset("clarin-knext/fiqa-pl", "corpus")["corpus"]

fiqa_corpus = fiqa_corpus["text"]
fiqa_idx = choice(len(fiqa_corpus), 100, replace=False)
fiqa_corpus = [fiqa_corpus[i] for i in fiqa_idx]

---

### NER baseline

In [6]:
import spacy

nlp = spacy.load("pl_core_news_sm")


def get_ents(text):
    doc = nlp(text)
    entity_dict = {}
    for ent in doc.ents:
        text, label = ent.text, ent.label_
        if (text, label) not in entity_dict.keys():
            entity_dict[(text, label)] = 0
        entity_dict[(text, label)] += 1
    return entity_dict

In [7]:
prompt = (
    "Chcę, abyś wymienił encje nazwane w polskim tekście, który Ci podam, i przypisał je do jednej z 6 kategorii."
    "Encje nazwane (eng. named entities) określają wybraną charakterystyczną jednostkę za pomocą zdefiniowanej nazwy."
    "6 kategorii, do których powinieneś przypisać encje nazwane: \n"
    "date - format daty/czasu: precyzyjna data, rok, itp. np. 11 Września. \n"
    "geogName - charakterystyczne obiekty geograficzne: rzeki, góry, itp. np. Wisła, Giewont. \n"
    "orgName - pełne nazwy organizacji: nazwy firm, itp. np. Microsoft, Bank PKO. "
    "persName - znane osoby: politycy, celebryci, postacie historyczne itp. np. Artur Rojek, Papież Franciszek. \n"
    "placeName - jednostki polityczne: miasta, państwa, itd. np. Kraków, Czechy, Australia, Bojszowy Nowe. \n"
    "time - Godzina. np. 18:30. \n"
    "Zignoruj fragmenty tekstu nie będące charakterystycznymi encjami nazwanymi. "
    "Wynik powinien być listą krotek. Każda krotka powinna składać się z encji nazwanej i odpowiadającej jej kategorii (jednej z wymienionych). "
    "Przykładowy format poprawnego wyniku dla tekstu: \n"
    '"Ostatnim razem w górach izerskich byłem 11 września": [("góry izerskie", "geogName"), ("11 września", "date")] \n'
    '"Nigdy nie byłem w stanach zjednoczonych, ale wiem kto to Barack Obama": [("Stany Zjednoczone", "placeName"), ("Barack Obama", "persName")] \n'
    "Nie dodawaj żadnych dodatkowych informacji, wyjaśnień, ani kodu w swojej odpowiedzi. Zwróć jedynie poprawną listą języka Python. "
    "Tekst: "
)

In [8]:
import re


def extract_list(text):
    regex = r"\[[^\[\]]*\]"
    return re.findall(regex, text)

In [9]:
from ast import literal_eval


# We use spacy to lemmatize if possible as the llm seems to be oblivious to the intricacy of case declension
def get_llm_ents(text):
    response = chat_ollama(prompt + text)

    print(response)

    lists = extract_list(response)
    if not lists or len(lists[0]) == 0:
        return {}
    response = lists[0]

    entity_dict = {}
    response = literal_eval(response)
    if len(response) == 0 or len(response[0]) != 2:
        return {}
    for entity, label in response:
        ents_tmp = list(nlp(entity).sents)
        if len(ents_tmp):
            lemma = " ".join([l.lemma_ for l in ents_tmp[0]])
        else:
            lemma = entity
        if (lemma, label) not in entity_dict.keys():
            entity_dict[(lemma, label)] = 0
        entity_dict[(lemma, label)] += 1
    return entity_dict

In [10]:
example_text = "Wczoraj w Krakowie miało miejsce spotkanie prezydentów Polski i Stanów Zjednoczonych. Polacy cieszyli się z wizyty Michaela Jacksona"

get_llm_ents(example_text)

[("Kraków", "placeName"), ("Polska", "placeName"), ("Stany Zjednoczone", "placeName"), ("Michael Jackson", "persName")]


{('Kraków', 'placeName'): 1,
 ('Polska', 'placeName'): 1,
 ('Stany Zjednoczone', 'placeName'): 1,
 ('Michael Jackson', 'persName'): 1}

---

### Compare spaCy to Bielik

I gave up on trying to run it without providing the examples.

The returned results were so bad that parsing the outputs was practically impossible.

It's safe to conclude that few shot learning can drastically improve the quality of the response.

In [11]:
from pathlib import Path
import pickle

save_path = Path("data")
save_path.mkdir(exist_ok=True, parents=True)

In [12]:
from tqdm import tqdm

if not (save_path / "fiqa_spacy.pkl").exists():
    fiqa_spacy = [get_ents(text) for text in tqdm(fiqa_corpus)]

    with open(save_path / "fiqa_spacy.pkl", "wb") as fp:
        pickle.dump(fiqa_spacy, fp)

with open(save_path / "fiqa_spacy.pkl", "rb") as fp:
    fiqa_spacy = pickle.load(fp)

In [13]:
if not (save_path / "fiqa_llm.pkl").exists():
    fiqa_llm = []
    for text in tqdm(fiqa_corpus):
        fiqa_llm.append(get_llm_ents(text))

    with open(save_path / "fiqa_llm.pkl", "wb") as fp:
        pickle.dump(fiqa_llm, fp)

with open(save_path / "fiqa_llm.pkl", "rb") as fp:
    fiqa_llm = pickle.load(fp)

In [15]:
def get_ent_cat_count(entity_dict_list):
    entities, categories = {}, {}
    for d in entity_dict_list:
        for (entity, category), count in d.items():
            if entity not in entities.keys():
                entities[entity] = 0
            if category not in categories.keys():
                categories[category] = 0
            entities[entity] += count
            categories[category] += count
    entities = sorted(list(entities.items()), key=lambda x: x[1], reverse=True)
    categories = sorted(list(categories.items()), key=lambda x: x[1], reverse=True)
    return entities, categories

In [16]:
spacy_entities, spacy_categories = get_ent_cat_count(fiqa_spacy)
llm_entities, llm_categories = get_ent_cat_count(fiqa_llm)

In [17]:
spacy_categories

[('orgName', 141),
 ('placeName', 70),
 ('persName', 66),
 ('date', 18),
 ('geogName', 16)]

In [18]:
spacy_entities[:25]

[('CFA', 23),
 ('MBA', 16),
 ('USA', 12),
 ('Google', 5),
 ('HSA', 5),
 ('Indiach', 4),
 ('SS', 4),
 ('LLC', 4),
 ('eBay', 3),
 ('Forex', 3),
 ('IRS', 3),
 ('SIP', 3),
 ('Amazon', 3),
 ('CFD', 3),
 ('Apple', 3),
 ('Sydney', 3),
 ('CBV', 3),
 ('Stanach Zjednoczonych', 2),
 ('TPS', 2),
 ('amerykańskimi', 2),
 ('Wielkiej Brytanii', 2),
 ('BB', 2),
 ('Columbia', 2),
 ('Whartona', 2),
 ('Aptery', 2)]

In [19]:
llm_categories

[('orgName', 123),
 ('date', 55),
 ('placeName', 32),
 ('time', 17),
 ('persName', 15),
 ('geogName', 6),
 ('service', 4),
 ('currency', 2),
 ('', 2),
 ('object', 2),
 ('number', 1)]

In [20]:
llm_entities[:25]

[('1987', 33),
 ('Stany Zjednoczone', 7),
 ('wielki Brytania', 3),
 ('Barack Obama', 2),
 ('Amazon', 2),
 ('USA', 2),
 ('rezerwa Federalny', 2),
 ('UPS', 2),
 ('USPS', 2),
 ('fałszywy równoważność', 1),
 ('być prawie pewny', 1),
 ('dane', 1),
 ('lewicow źródło', 1),
 ('przeciwnik', 1),
 ('naukowy', 1),
 ('Mindwin', 1),
 ('rynek rolny', 1),
 ('rynek towarowy', 1),
 ('uniwersytet', 1),
 ('kawiarnia', 1),
 ('bank godzina', 1),
 ('Fort Lauderdale', 1),
 ('elektrownia na paliwo kopalny', 1),
 ('Wall Street', 1),
 ('Brexit', 1)]

We can see that the results of llm retrieval is horrible in comparison to spaCy's ner. The model hallucinated several new classes and has numerous false positives.

In [58]:
import pandas as pd

df = pd.read_csv("annotations.csv")

In [59]:
class_map = {
    "PER": "persName",
    "LOC": "placeName",
    "ORG": "orgName",
    "GPE": "geogName",
    "DATE": "date",
    "TIME": "time",
}


def extract_from_annotation(annotation):
    entity_dict = {}
    for entity, category, _ in annotation:
        ents_tmp = list(nlp(entity).sents)
        if len(ents_tmp):
            entity = " ".join([l.lemma_ for l in ents_tmp[0]])

        category = class_map[category]
        if (entity, category) not in entity_dict.keys():
            entity_dict[(entity, category)] = 0
        entity_dict[(entity, category)] += 1
    return entity_dict

In [60]:
texts = df["Tekst"]
entities = [extract_from_annotation(literal_eval(e)) for e in df["Encje"]]

In [61]:
get_ents(texts[0])

{('bokserski', 'placeName'): 1,
 ('AT&T Stadium', 'geogName'): 1,
 ('Arlington', 'placeName'): 1,
 ('Teksasie Jake Paul', 'placeName'): 1,
 ("Mike'a Tysona", 'persName'): 1,
 ('Sylvester Stallone', 'persName'): 1}

In [62]:
from tqdm import tqdm

if not (save_path / "predictions.pkl").exists():
    predictions = []

    for text in tqdm(texts):
        predictions.append(get_llm_ents(text))

    with open(save_path / "predictions.pkl", "wb") as fp:
        pickle.dump(predictions, fp)

with open(save_path / "predictions.pkl", "rb") as fp:
    predictions = pickle.load(fp)

In [63]:
def get_TP_FP_FN(y_test, y_pred, use_class=False):
    if not use_class:
        y_test = {k[0]: v for k, v in y_test.items()}
        y_pred = {k[0]: v for k, v in y_pred.items()}

    TP = FP = FN = 0

    for e in y_pred:
        if e in y_test:
            pred = y_pred[e]
            test = y_test[e]
            if pred == test:
                TP += pred
            elif pred > test:
                TP += test
                FP += pred - test
            else:
                TP += pred
                FN += test - pred
        else:
            FP += y_pred[e]

    for e in y_test:
        if e not in y_pred:
            FN += y_test[e]

    return TP, FP, FN

In [64]:
scores = [get_TP_FP_FN(t, p) for t, p in zip(entities, predictions)]

In [65]:
TP = sum([scr[0] for scr in scores])
FP = sum([scr[1] for scr in scores])
FN = sum([scr[2] for scr in scores])

In [66]:
TP, FP, FN

(58, 11, 36)

In [67]:
prec = TP / (TP + FP)
rec = TP / (TP + FN)
f1 = 2 * (prec * rec) / (prec + rec)

In [68]:
print(f"Precission : {prec}")
print(f"Recall     : {rec}")
print(f"f1 score   : {f1}")

Precission : 0.8405797101449275
Recall     : 0.6170212765957447
f1 score   : 0.7116564417177915


The model skips around 40% of entities it should be detecting but it only 15% of it's predictions are false. 

I'm surprised by how well it worked

Let's check the same scores for spaCy

In [69]:
# this function converts found annotation to normalized form
# (same one as the grand truth - extract_from_annotation function above)
# Some words are detected and not declined
# We want to make sure that the same entity is returned in the same way


def normalize_entity_dict(annotation):
    entity_dict = {}
    for (entity, category), _ in annotation.items():
        ents_tmp = list(nlp(entity).sents)
        if len(ents_tmp):
            entity = " ".join([l.lemma_ for l in ents_tmp[0]])

        if (entity, category) not in entity_dict.keys():
            entity_dict[(entity, category)] = 0
        entity_dict[(entity, category)] += 1
    return entity_dict

In [70]:
predictions_spacy = [normalize_entity_dict(get_ents(t)) for t in texts]

In [71]:
scores = [get_TP_FP_FN(t, p) for t, p in zip(entities, predictions_spacy)]

TP = sum([scr[0] for scr in scores])
FP = sum([scr[1] for scr in scores])
FN = sum([scr[2] for scr in scores])

TP, FP, FN

(54, 34, 40)

In [72]:
prec = TP / (TP + FP)
rec = TP / (TP + FN)
f1 = 2 * (prec * rec) / (prec + rec)

print(f"Precission : {prec}")
print(f"Recall     : {rec}")
print(f"f1 score   : {f1}")

Precission : 0.6136363636363636
Recall     : 0.574468085106383
f1 score   : 0.5934065934065934


We can see the scores are worse! Let us compare on an example

In [73]:
print(texts[0])
print()
print(entities[0])
print()
print(predictions[0])
print()
print(predictions_spacy[0])

Niemal dwa tygodnie temu doszło do walki, na którą czekał cały bokserski świat. Na AT&T Stadium w Arlington w Teksasie Jake Paul pokonał Mike'a Tysona po jednogłośnej decyzji sędziów. Część kibiców i ekspertów twierdzi jednak, że pojedynek... mógł być ustawiony. Do tego grona najwyraźniej należy Sylvester Stallone, który po ogłoszeniu werdyktu wygłosił dosyć kontrowersyjną opinię. Teraz popularny aktor postanowił przeprosić za swoje słowa.

{('jake Paul', 'persName'): 1, ('Mike Tyson', 'persName'): 1, ('sylvester Stallone', 'persName'): 1, ('aT&T stadium', 'orgName'): 1, ('Arlington', 'geogName'): 1, ('Teksas', 'geogName'): 1}

{('jake Paul', 'persName'): 1, ('Mike Tyson', 'persName'): 1, ('aT&T stadium', 'orgName'): 1, ('Arlington', 'placeName'): 1, ('Teksas', 'placeName'): 1}

{('bokserski', 'placeName'): 1, ('aT&T stadium', 'geogName'): 1, ('Arlington', 'placeName'): 1, ('Teksasie jake Paul', 'placeName'): 1, ('Mike Tysona', 'persName'): 1, ('sylvester Stallone', 'persName'): 1}


We can see that the LLM proved to work better than spacy! It's a surprising result. Seeing the performance of LLM on fiqa dataset we could see that it detected a lot of incorrect entities.

It's possible that hand annotated grand-truth sentences are easier for the model to work with. They are also shorter, and that makes the context much simpler.

In conclusion. I'd lean towards using spacy when I need to process a lot of data or long texts. It's much faster. If I want precise answers for simpler or shorter texts I'd consider using an LLM instead.


### Questions

1. How does the performance of LLM-based NER compare to traditional approaches? What are the trade-offs in terms of accuracy, speed, and resource usage?
    - The LLM-based NER works better on simple data. However, it's much slower and might not be great for larger tasks
2. Which prompting strategy proved most effective for NER and classification tasks? Why?
    - LLM-based NER needed examples to work properly and return even remotely acceptable output. Without it even the format of the output string was incorrect.
3. What are the limitations and potential biases of using LLMs for NER and classification?
    - LLM are very slow and inaccurate for longer texts. Long text would need a more accurate and probably larger model. It can be expensive and resource-consuming.
4. In what scenarios would you recommend using traditional NER vs. LLM-based approaches?
    - Described above